# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [41]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [9]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [10]:
from mysecrets import IEX_CLOUD_API_TOKEN



## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [54]:
symbol = 'ABMD'

api_url = f'https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

data

[{'beta': None,
  'totalCash': 820372000,
  'currentDebt': 134060000,
  'revenue': 1074096000,
  'grossProfit': 873506000,
  'totalRevenue': 1074096000,
  'EBITDA': 281669000,
  'revenuePerShare': 23.82,
  'revenuePerEmployee': 783439.82,
  'debtToEquity': 1.1018043404704676,
  'profitMargin': 0.2483614127601257,
  'enterpriseValue': None,
  'enterpriseValueToRevenue': None,
  'priceToSales': None,
  'priceToBook': None,
  'forwardPERatio': None,
  'pegRatio': None,
  'peHigh': 82.36364523685567,
  'peLow': 50.07768252212382,
  'week52highDate': None,
  'week52lowDate': None,
  'putCallRatio': 3.648128342245989,
  'companyName': 'Abiomed Inc.',
  'marketcap': None,
  'week52high': None,
  'week52low': None,
  'week52highSplitAdjustOnly': None,
  'week52highDateSplitAdjustOnly': None,
  'week52lowSplitAdjustOnly': None,
  'week52lowDateSplitAdjustOnly': None,
  'week52change': None,
  'sharesOutstanding': 45091184,
  'float': None,
  'avg10Volume': None,
  'avg30Volume': None,
  'day200

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [76]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))



symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame( columns = my_columns)

    



Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [77]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    batch_basic_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    basic_data = requests.get(batch_basic_api_call_url).json()
    for i in range(0,len(data)):
        final_dataframe = final_dataframe._append(
            pd.Series(
            [
                basic_data[i]["symbol"],
                basic_data[i]["latestPrice"],
                data[i]["year1ChangePercent"],
                0
            ], 
            index = my_columns
            ),ignore_index = True
            )


for row in final_dataframe.index:
    if math.isnan(final_dataframe.loc[row, 'One-Year Price Return']):
        final_dataframe = final_dataframe.drop(labels=row , axis=0)


final_dataframe
        


        

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/3440354964.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = final_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/3440354964.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = final_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/3440354964.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.68,-0.107415,0
1,AAL,14.38,-0.015743,0
2,AAP,61.39,-0.588306,0
3,AAPL,185.14,0.434282,0
4,ABBV,162.31,0.044582,0
...,...,...,...,...
500,YUM,128.22,0.006724,0
501,ZBH,121.87,-0.025659,0
502,ZBRA,256.44,-0.081815,0
503,ZION,44.04,-0.093596,0


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [23]:
final_dataframe.sort_values('One-Year Price Return' , ascending= False , inplace= True)
final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(inplace= True)
final_dataframe

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/3284406488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('One-Year Price Return' , ascending= False , inplace= True)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,345,NVDA,522.5300,2.518150,0
1,1,275,LB,79.9200,2.281287,0
2,2,31,AMD,146.1800,1.285491,0
3,3,390,RCL,124.1200,1.257138,0
4,4,37,ANET,245.8500,1.186305,0
5,5,370,PHM,103.7300,1.166630,0
6,6,208,GPS,21.6600,0.938671,0
7,7,339,NOW,696.2600,0.899599,0
8,8,50,AVGO,1074.8500,0.886800,0
9,9,84,CCL,17.3500,0.885870,0


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [28]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    try:
        float(portfolio_size)
    except ValueError:
        print('This is not a number. \nPlease try again!')
        portfolio_size = input('Enter the size of your portfolio')

portfolio_input()
portfolio_size
    

Enter the size of your portfolio 10000000


'10000000'

In [31]:
position_size = float(portfolio_size)/len(final_dataframe.index)


column_shares_to_buy = 'Number of Shares to Buy'
column_price = 'Price'

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, column_shares_to_buy] = math.floor(position_size/final_dataframe.loc[i,column_price])


final_dataframe

,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,345,NVDA,522.5300,2.518150,382
1,1,275,LB,79.9200,2.281287,2502
2,2,31,AMD,146.1800,1.285491,1368
3,3,390,RCL,124.1200,1.257138,1611
4,4,37,ANET,245.8500,1.186305,813
5,5,370,PHM,103.7300,1.166630,1928
6,6,208,GPS,21.6600,0.938671,9233
7,7,339,NOW,696.2600,0.899599,287
8,8,50,AVGO,1074.8500,0.886800,186
9,9,84,CCL,17.3500,0.885870,11527


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [105]:
column_ticker = 'Ticker'
column_1_yr_price_return = 'One-Year Price Return'
column_1_yr_return_percentile = 'One-Year Return Percentile'
column_6_mnts_price_return = 'Six-Months Price Return'
column_6_mnts_return_percentile = 'Six-Months Return Percentile'
column_3_mnts_price_return = 'Three-Months Price Return'
column_3_mnts_return_percentile = 'Three-Months Return Percentile'
column_1_mnt_price_return = 'One-Month Price Return'
column_1_mnt_return_percentile = 'One-Month Return Percentile'
column_HQM_Score = 'HQM Score'

hqm_columns = [column_ticker, column_price, column_shares_to_buy,
              column_1_yr_price_return, column_1_yr_return_percentile,
              column_6_mnts_price_return, column_6_mnts_return_percentile,
              column_3_mnts_price_return, column_3_mnts_return_percentile,
              column_1_mnt_price_return, column_1_mnt_return_percentile,
               column_HQM_Score
              ]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/CORE/ADVANCED_STATS/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    batch_basic_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    basic_data = requests.get(batch_basic_api_call_url).json()
    for i in range(0,len(basic_data)):
        hqm_dataframe = hqm_dataframe._append(
            pd.Series(
                [
                    basic_data[i]["symbol"],
                    basic_data[i]["latestPrice"],
                    'N/A',
                    data[i]["year1ChangePercent"],
                    'N/A',
                    data[i]["month6ChangePercent"],
                    'N/A',
                    data[i]["month3ChangePercent"],
                    'N/A',
                    data[i]["month1ChangePercent"],
                    'N/A' ,
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index= True
        )


for row in hqm_dataframe.index:
    if math.isnan(hqm_dataframe.loc[row, 'One-Year Price Return']):
        hqm_dataframe = hqm_dataframe.drop(labels=row , axis=0)
    
hqm_dataframe

/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/2119643519.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = hqm_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/2119643519.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = hqm_dataframe._append(
/var/folders/tt/ptcp60n908q2mwfffgwm6jlm0000gn/T/ipykernel_10983/2119643519.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entri

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.300,N/A,-0.107415,N/A,0.121068,N/A,0.176321,N/A,0.029096,N/A,N/A
1,AAL,14.360,N/A,-0.015743,N/A,-0.216349,N/A,0.174837,N/A,0.045058,N/A,N/A
2,AAP,61.010,N/A,-0.588306,N/A,-0.107674,N/A,0.201147,N/A,0.090990,N/A,N/A
3,AAPL,185.785,N/A,0.434282,N/A,-0.023864,N/A,0.037086,N/A,-0.054008,N/A,N/A
4,ABBV,164.395,N/A,0.044582,N/A,0.223262,N/A,0.099421,N/A,0.087286,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.370,N/A,0.006724,N/A,-0.030124,N/A,0.083395,N/A,0.031205,N/A,N/A
501,ZBH,123.370,N/A,-0.025659,N/A,-0.131207,N/A,0.103199,N/A,0.041032,N/A,N/A
502,ZBRA,255.390,N/A,-0.081815,N/A,-0.131360,N/A,0.152125,N/A,0.086012,N/A,N/A
503,ZION,44.010,N/A,-0.093596,N/A,0.556019,N/A,0.312214,N/A,0.131261,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [106]:
time_series = ["One-Year", "Six-Months", "Three-Months","One-Month"]






for row in hqm_dataframe.index:
    for time_period in time_series:
        change_return = f'{time_period} Price Return'
        change_col = f'{time_period} Return Percentile'
        # print(change_return)
        # print(change_col)
        # print(stats.percentileofscore(hqm_dataframe[change_return], hqm_dataframe.loc[row,change_return]))
        hqm_dataframe.loc[row, change_col] = stats.percentileofscore(hqm_dataframe[change_return], hqm_dataframe.loc[row,change_return])/100


hqm_dataframe



,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.300,N/A,-0.107415,0.174538,0.121068,0.652977,0.176321,0.716632,0.029096,0.38809,N/A
1,AAL,14.360,N/A,-0.015743,0.328542,-0.216349,0.028747,0.174837,0.708419,0.045058,0.566735,N/A
2,AAP,61.010,N/A,-0.588306,0.004107,-0.107674,0.110883,0.201147,0.792608,0.090990,0.831622,N/A
3,AAPL,185.785,N/A,0.434282,0.880903,-0.023864,0.252567,0.037086,0.227926,-0.054008,0.043121,N/A
4,ABBV,164.395,N/A,0.044582,0.455852,0.223262,0.856263,0.099421,0.441478,0.087286,0.817248,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.370,N/A,0.006724,0.38809,-0.030124,0.238193,0.083395,0.37577,0.031205,0.425051,N/A
501,ZBH,123.370,N/A,-0.025659,0.310062,-0.131207,0.086242,0.103199,0.457906,0.041032,0.517454,N/A
502,ZBRA,255.390,N/A,-0.081815,0.209446,-0.131360,0.084189,0.152125,0.620123,0.086012,0.809035,N/A
503,ZION,44.010,N/A,-0.093596,0.184805,0.556019,0.99384,0.312214,0.924025,0.131261,0.948665,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [107]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_series:
        column_name = f'{time_period} Return Percentile'
        momentum_percentiles.append(hqm_dataframe.loc[row,column_name])

    hqm_dataframe.loc[row, column_HQM_Score] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,130.300,N/A,-0.107415,0.174538,0.121068,0.652977,0.176321,0.716632,0.029096,0.38809,0.48306
1,AAL,14.360,N/A,-0.015743,0.328542,-0.216349,0.028747,0.174837,0.708419,0.045058,0.566735,0.408111
2,AAP,61.010,N/A,-0.588306,0.004107,-0.107674,0.110883,0.201147,0.792608,0.090990,0.831622,0.434805
3,AAPL,185.785,N/A,0.434282,0.880903,-0.023864,0.252567,0.037086,0.227926,-0.054008,0.043121,0.351129
4,ABBV,164.395,N/A,0.044582,0.455852,0.223262,0.856263,0.099421,0.441478,0.087286,0.817248,0.64271
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.370,N/A,0.006724,0.38809,-0.030124,0.238193,0.083395,0.37577,0.031205,0.425051,0.356776
501,ZBH,123.370,N/A,-0.025659,0.310062,-0.131207,0.086242,0.103199,0.457906,0.041032,0.517454,0.342916
502,ZBRA,255.390,N/A,-0.081815,0.209446,-0.131360,0.084189,0.152125,0.620123,0.086012,0.809035,0.430698
503,ZION,44.010,N/A,-0.093596,0.184805,0.556019,0.99384,0.312214,0.924025,0.131261,0.948665,0.762834


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [108]:
hqm_dataframe.sort_values(column_HQM_Score, ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AMD,148.660,N/A,1.219810,0.995893,0.318901,0.944559,0.395344,0.971253,0.157772,0.979466,0.972793
1,INTC,47.665,N/A,0.696124,0.967146,0.529519,0.989733,0.341280,0.948665,0.125293,0.934292,0.959959
2,PVH,122.965,N/A,0.525260,0.928131,0.445877,0.979466,0.623503,0.997947,0.116756,0.915811,0.955339
3,AVGO,1078.665,N/A,0.938223,0.98768,0.297213,0.928131,0.276423,0.899384,0.156904,0.977413,0.948152
4,DHI,154.450,N/A,0.650997,0.960986,0.320962,0.946612,0.433573,0.983573,0.096057,0.858316,0.937372
5,PHM,105.370,N/A,1.165928,0.99384,0.382423,0.967146,0.415646,0.979466,0.074779,0.753593,0.923511
6,ANET,250.810,N/A,1.134587,0.989733,0.551899,0.991786,0.248473,0.856263,0.094496,0.852156,0.922485
7,COST,672.350,N/A,0.477280,0.901437,0.339077,0.954825,0.253078,0.86653,0.142587,0.960986,0.920945
8,COF,129.370,N/A,0.395698,0.87269,0.244094,0.889117,0.417341,0.98152,0.125667,0.938398,0.920431
9,NRG,51.320,N/A,0.651689,0.963039,0.382843,0.969199,0.314515,0.928131,0.081782,0.794661,0.913758


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [109]:
portfolio_input()

Enter the size of your portfolio 10000000


In [110]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, column_shares_to_buy] = math.floor(position_size/hqm_dataframe.loc[row, column_price])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AMD,148.660,1345,1.219810,0.995893,0.318901,0.944559,0.395344,0.971253,0.157772,0.979466,0.972793
1,INTC,47.665,4195,0.696124,0.967146,0.529519,0.989733,0.341280,0.948665,0.125293,0.934292,0.959959
2,PVH,122.965,1626,0.525260,0.928131,0.445877,0.979466,0.623503,0.997947,0.116756,0.915811,0.955339
3,AVGO,1078.665,185,0.938223,0.98768,0.297213,0.928131,0.276423,0.899384,0.156904,0.977413,0.948152
4,DHI,154.450,1294,0.650997,0.960986,0.320962,0.946612,0.433573,0.983573,0.096057,0.858316,0.937372
5,PHM,105.370,1898,1.165928,0.99384,0.382423,0.967146,0.415646,0.979466,0.074779,0.753593,0.923511
6,ANET,250.810,797,1.134587,0.989733,0.551899,0.991786,0.248473,0.856263,0.094496,0.852156,0.922485
7,COST,672.350,297,0.477280,0.901437,0.339077,0.954825,0.253078,0.86653,0.142587,0.960986,0.920945
8,COF,129.370,1545,0.395698,0.87269,0.244094,0.889117,0.417341,0.98152,0.125667,0.938398,0.920431
9,NRG,51.320,3897,0.651689,0.963039,0.382843,0.969199,0.314515,0.928131,0.081782,0.794661,0.913758


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [111]:
writer = pd.ExcelWriter('momentum_strategy.xlsx' , engine='xlsxwriter' )

hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy',index=False)




## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [112]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [113]:
column_formats = {
    'A' :[column_ticker , string_template],
    'B' :[column_price , dollar_template],
    'C' : [column_shares_to_buy,  integer_template],
    'D':   [column_1_yr_price_return,  percent_template],
    'E' : [column_1_yr_return_percentile,   percent_template],
     'F':  [column_6_mnts_price_return,   percent_template],
    'G' :  [column_6_mnts_return_percentile,  percent_template],
     'H':   [ column_3_mnts_price_return,   percent_template],
    'I' : [column_3_mnts_return_percentile,   percent_template],
     'J':   [column_1_mnt_price_return,   percent_template],
    'K' :  [column_1_mnt_return_percentile,   percent_template],
    'L' :    [column_HQM_Score,  percent_template]
              
    
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,  column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])

writer.close()

## 
Saving Our Excel Output

As before, saving our Excel output is very easy: